In [1]:
# imports
import os
from os import path
import re
import glob as glob
import shutil as shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# to print full traceback in cought exceptions
import traceback

In [2]:
# regions from 1 to 9
regions = [
    "Burgenland",
    "Kärnten",
    "Niederösterreich",
    "Oberösterreich",
    "Salzburg",
    "Steiermark",
    "Tirol",
    "Vorarlberg",
    "Wien"
]

region_dirs = ["ehyd_messstellen_{0}_gw_{1}".format(i + 1, regions[i]) for i in range(0,len(regions))]
region_dirs

['ehyd_messstellen_1_gw_Burgenland',
 'ehyd_messstellen_2_gw_Kärnten',
 'ehyd_messstellen_3_gw_Niederösterreich',
 'ehyd_messstellen_4_gw_Oberösterreich',
 'ehyd_messstellen_5_gw_Salzburg',
 'ehyd_messstellen_6_gw_Steiermark',
 'ehyd_messstellen_7_gw_Tirol',
 'ehyd_messstellen_8_gw_Vorarlberg',
 'ehyd_messstellen_9_gw_Wien']

# Create a DF for all Measuring Points
### - Can be useful for data aggregation on geographical base -

In [3]:
def get_num_from_str(string):
    matches = re.findall("\d+\.\d+", string)
    if len(matches):
        return matches[0]
    else:
        return np.nan

def get_mp_attrs(region, region_dir, mp_num):

    mp_base_data_path = path.join(region_dir, "Stammdaten", "Stammdaten-{0}.txt".format(mp_num))
    
    land_height = np.nan
    mp_height = np.nan
    bottom_line = np.nan
    t_measuring_depth = np.nan
    
    with open(mp_base_data_path, encoding='unicode_escape') as f:
        for num, line in enumerate(f, 1):
            if "Geländehöhe" in line:
                land_height = get_num_from_str(line)
                
            elif "Messpunkthöhe" in line:
                mp_height = get_num_from_str(line)
                
            elif "Sohllage" in line:
                bottom_line = get_num_from_str(line)
                
            elif "T-Messtiefe u.GOK" in line:
                t_measuring_depth = get_num_from_str(line)
                
                break # since it's always listed latest in the file
    
    mp = {
        "land_height": land_height,
        "mp_height": mp_height,
        "bottom_line": bottom_line,
        "t_measuring_depth": t_measuring_depth
    }
    
    return mp


# test
region_idx = 0

# try mp with full attr
mp_num = 305714
mp_attrs = get_mp_attrs(regions[region_idx], region_dirs[region_idx], mp_num = mp_num)
print("mp {0} attrs:".format(mp_num), mp_attrs)

# try mp with missing attr -> np.nan in missed value
mp_num = 300137
mp_attrs = get_mp_attrs(regions[region_idx], region_dirs[region_idx], mp_num = mp_num)
print("mp {0} attrs:".format(mp_num), mp_attrs)

mp 305714 attrs: {'land_height': '121.76', 'mp_height': '122.76', 'bottom_line': '111.76', 't_measuring_depth': '8.00'}
mp 300137 attrs: {'land_height': '154.76', 'mp_height': '154.76', 'bottom_line': '151.36', 't_measuring_depth': nan}


In [4]:
values = []

for i in range(len(regions)):
    region = regions[i]
    region_dir = region_dirs[i]
    # mps: measurment points 
    df_mps_path = path.join(region_dir, "messstellen_alle.csv")
    df_mps = pd.read_csv(df_mps_path, sep=";")
    # filter to typ == 'gw' then del typ col
    df_mps = df_mps.query("typ=='gw'")
    del df_mps["typ"]
    
    df_mps["region"] = region
    
    # for every mp in the region get attributes
    # create attrs cols and init with np.nan
    mp_attrs = ["land_height", "mp_height", "bottom_line", "t_measuring_depth"]
    for mp_attr in mp_attrs:
        df_mps[mp_attr] = np.nan
    
    # fill with values when available
    for index, row in df_mps.iterrows():
        mp_num = row["hzbnr01"]
        mp_attr_vals = get_mp_attrs(region, region_dir, mp_num = mp_num)
        for mp_attr in mp_attrs:
            df_mps.at[index, mp_attr] = mp_attr_vals[mp_attr]
    
    values.extend(df_mps.values.tolist())

colnames = ["x", "y", "dbmsnr", "hzbnr01", "region"] + mp_attrs
df_mps_all = pd.DataFrame(values, columns = colnames)
df_mps_all["x"] = df_mps_all["x"].str.replace(",", ".").astype(float)
df_mps_all["y"] = df_mps_all["y"].str.replace(",", ".").astype(float)

df_mps_all

/tmp/ipykernel_7499/3722866313.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '125.76' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mps.at[index, mp_attr] = mp_attr_vals[mp_attr]
/tmp/ipykernel_7499/3722866313.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '126.66' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mps.at[index, mp_attr] = mp_attr_vals[mp_attr]
/tmp/ipykernel_7499/3722866313.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '108.29' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mps.at[index, mp_attr] = mp_attr_vals[mp_attr]
/tmp/ipykernel_7499/3722866313.py:26: FutureWarning:

/tmp/ipykernel_7499/3722866313.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '157.89' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mps.at[index, mp_attr] = mp_attr_vals[mp_attr]
/tmp/ipykernel_7499/3722866313.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '158.89' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mps.at[index, mp_attr] = mp_attr_vals[mp_attr]
/tmp/ipykernel_7499/3722866313.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '147.50' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mps.at[index, mp_attr] = mp_attr_vals[mp_attr]
/tmp/ipykernel_7499/3722866313.py:26: FutureWarning:

,x,y,dbmsnr,hzbnr01,region,land_height,mp_height,bottom_line,t_measuring_depth
0,679020.30,448967.00,1002266,335588,Burgenland,125.76,126.66,108.29,NaN
1,643311.78,398158.79,1002285,335810,Burgenland,235.69,236.59,225.74,10.00
2,616039.28,341145.21,1002422,345876,Burgenland,234.06,234.86,228.06,NaN
3,615420.17,341879.22,1002237,335299,Burgenland,238.18,238.28,233.38,NaN
4,665124.60,430698.99,1002327,345256,Burgenland,116.99,117.54,114.65,NaN
...,...,...,...,...,...,...,...,...,...
3786,631609.35,485861.88,9002133,319749,Wien,157.12,157.07,142.07,NaN
3787,622685.15,480101.35,9002268,350074,Wien,187.09,187.94,175.89,NaN
3788,633723.06,481067.86,9002260,350116,Wien,160.93,161.93,150.93,NaN
3789,633459.50,482115.68,9002231,341438,Wien,154.76,155.71,145.15,6.05


In [5]:
# Save to .csv
df_mps_all.to_csv("df_mps_all.csv", index=False)
print("> df_mps_all Saved as .csv!")

> df_mps_all Saved as .csv!


In [6]:
# Use util fun to find all mps in a selected radius
from utils_mps import find_mps_in_radius

hzbnr01 = 335588
# TODO: fine-tuning radius value to aggregate data for model training
radius = 5000
df_mps_rds = find_mps_in_radius(df_mps_all, hzbnr01, radius)
df_mps_rds

,x,y,dbmsnr,hzbnr01,region,land_height,mp_height,bottom_line,t_measuring_depth
0,679020.30,448967.00,1002266,335588,Burgenland,125.76,126.66,108.29,NaN
20,676735.27,448176.78,1002166,326074,Burgenland,129.25,129.95,117.80,11.12
56,675621.69,449296.94,1002176,326173,Burgenland,144.56,145.41,132.86,NaN
94,676006.48,445085.10,1002167,326082,Burgenland,122.92,123.37,114.65,6.10
121,678066.41,450372.91,1002250,335422,Burgenland,151.60,152.50,124.33,27.00
160,680848.55,452791.43,1002262,335547,Burgenland,126.94,127.74,108.76,6.20


# Process Single Regions

In [7]:
# know how many rows to skip in pd.read_csv
def csv_find_skiprows(filepath, lookup):
    skiprows = 0
    # first know how many rows to skip in pd.read_csv. TO do this will open the file and look for "Werte:"
    with open(filepath, encoding='unicode_escape') as f:
        for num, line in enumerate(f, 1):
            if lookup in line:
                skiprows = num
                # print("skiprows:", skiprows)
                break
                
    return skiprows

def process_region_sub_dir(region, region_dir, folder_name, val_col_name, debugging=False):
    
    folder_dir = path.join(region_dir, folder_name)
    
    print("> Processing {0} - {1}..".format(region, folder_name))
    
    # clean previous processing output
    output_dir = path.join(folder_dir, "processed_data")
    
    try:
        shutil.rmtree(output_dir)
    except:
        pass
    os.mkdir(output_dir)
    
    filenames = os.listdir(folder_dir)
    for filename in filenames:
        
        filepath = path.join(folder_dir, filename)
        
        # process only files (don't iterate over output dir)
        if not path.isfile(filepath):
            continue
        
        # first know how many rows to skip in pd.read_csv. TO do this will open the file and look for "Werte:"
        lookup = "Werte:"
        skiprows = csv_find_skiprows(filepath, lookup)
        
        # load and process .csv file
     
        df = pd.read_csv(filepath, sep=" ;", header=None, skiprows=skiprows,
                               encoding='unicode_escape')
        
        

        # manipulate data splitting values into more possible regressors (e.g. date -> day, month, year)
        df.columns = ["date & time", val_col_name, "empty"]
        del df["empty"]
        
        # remove rows with gaps ("Lücke")
        df = df[~df[val_col_name].str.contains("Lücke")]
        
        # clean data from white spaces
        df["date & time"] = df["date & time"].str.replace("  ", "")
        
        df[["date", "time"]] = df["date & time"].str.split(" ", expand=True)
        df[["day", "month", "year"]] = df["date"].str.split(".", expand=True)
        df[["hour", "minute", "second"]] = df["time"].str.split(":", expand=True)
        # clean gw-level value and convert it to float
        df[val_col_name] = df.apply(
            lambda row: float(row[val_col_name].split(" ;")[0].replace(",", "."))
            , axis=1
        )
        # rearrange cols
        df = df[["date", "day", "month", "year", "time", "hour", "minute", "second", val_col_name]]
        
        df.to_csv(path.join(output_dir, "processed_{0}".format(filename)), index=False)
        
        # if debugging, process only one file from each sub dir 
        if debugging:
            break
        
    print("\t- Done!")

    return None

In [12]:
def create_region_mps_summary(region, region_dir, debugging):
    
    summary_dir  = path.join(region_dir, "Summary")
    
    print("> Creating {0} Summary..".format(region))
    
    # clean previous processing output   
    try:
        shutil.rmtree(summary_dir)
    except:
        pass
    os.mkdir(summary_dir)
    
    try:
        # get processed monthly water level files as starting point
        prc_mnth_lvls_dir = path.join(region_dir, "Grundwasserstand-Monatsmittel", "processed_data")
        mnth_lvls_filenames = os.listdir(prc_mnth_lvls_dir)
        
        for mnth_lvls_filename in mnth_lvls_filenames:
            
            mp_num = int(mnth_lvls_filename.split(".")[0].split("-")[-1])
            
            if debugging:
                mp_num = 305540 # override mp_num value to choose a mp that we have temperature data for
                print("mp_num:", mp_num)
                
            df = pd.read_csv(path.join(prc_mnth_lvls_dir, mnth_lvls_filename), sep=",")
            
            # add extra fields, init them as NaN, then try to fill them with values
            # add field: yr_max
            df["yr_max"] = np.nan
            
            yr_max_fpath = path.join(region_dir, "Grundwasserstand-Jahresmaxima", "processed_data", "processed_Grundwasserstand-Jahresmaxima-{0}.csv".format(mp_num))
            
            if path.exists(yr_max_fpath):
                
                df_yr_max = pd.read_csv(yr_max_fpath, sep=",")[["year", "gw-level"]]
                
                # set yr as index and create dict for O(1) search
                # first drop rows with "year" val duplicates to avoid errors when creating index
                df_len_before_drop = len(df_yr_max)
                df_yr_max.drop_duplicates(subset=["year"], inplace=True)
                df_len_after_drop = len(df_yr_max)
                if not df_len_before_drop == df_len_after_drop:
                    n_dropped = df_len_before_drop - df_len_after_drop
                    print(f'\t* Grundwasserstand-Jahresmaxima dropped {n_dropped} duplicates in "year".')
                    
                df_yr_max.set_index("year", inplace=True)
                dct_yr_max = df_yr_max.to_dict('index')
                
                # impute missing yr_max values from monthly values
                # find nan_years: the ears for which we don't have yr_max value
                nan_years = [nan_year for nan_year in list(df.year.unique()) if nan_year not in dct_yr_max]
                for nan_year in nan_years:
                    yr_max_val = df.query("year==@nan_year")["gw-level"].max()
                    dct_yr_max[nan_year] = {"gw-level": yr_max_val}
                    
                df["yr_max"] = df.apply(
                    lambda row: dct_yr_max[row["year"]]["gw-level"] if row["year"] in dct_yr_max else np.nan
                    , axis=1)
                
                # free resources
                del df_yr_max
                del dct_yr_max
                
            # add field: yr_max_dist
            df["yr_max_dist"] = np.nan
            
            df["yr_max_dist"] = df.apply(
                lambda row: round(row["yr_max"] - row["gw-level"], 2) if not row["yr_max"] == np.nan else np.nan
                , axis=1)
                
            # add field: yr_min
            df["yr_min"] = np.nan
            
            yr_min_fpath = path.join(region_dir, "Grundwasserstand-Jahresminima", "processed_data", "processed_Grundwasserstand-Jahresminima-{0}.csv".format(mp_num))
            
            if path.exists(yr_min_fpath):
                
                df_yr_min = pd.read_csv(yr_min_fpath, sep=",")[["year", "gw-level"]]
                
                # set yr as index and create dict for O(1) search
                # first drop rows with "year" val duplicates to avoid errors when creating index
                df_len_before_drop = len(df_yr_min)
                df_yr_min.drop_duplicates(subset=["year"], inplace=True)
                df_len_after_drop = len(df_yr_min)
                if not df_len_before_drop == df_len_after_drop:
                    n_dropped = df_len_before_drop - df_len_after_drop
                    print(f'\t* Grundwasserstand-Jahresminima dropped {n_dropped} duplicates in "year".')
                df_yr_min.set_index("year", inplace=True)
                dct_yr_min = df_yr_min.to_dict('index')
                
                # impute missing yr_min values from monthly values
                # find nan_years: the ears for which we don't have yr_min value
                nan_years = [nan_year for nan_year in list(df.year.unique()) if nan_year not in dct_yr_min]
                for nan_year in nan_years:
                    yr_min_val = df.query("year==@nan_year")["gw-level"].min()
                    dct_yr_min[nan_year] = {"gw-level": yr_min_val}
                
                df["yr_min"] = df.apply(
                    lambda row: dct_yr_min[row["year"]]["gw-level"] if row["year"] in dct_yr_min else np.nan
                    , axis=1)                
                
                # free resources
                del df_yr_min
                del dct_yr_min
                
            # add field: yr_min_dist
            df["yr_min_dist"] = np.nan
            
            df["yr_min_dist"] = df.apply(
                lambda row: round(row["gw-level"] - row["yr_min"], 2) if not row["yr_min"] == np.nan else np.nan
                , axis=1)
                
            # add fields: yr_avg, yr_avg_dist, yr_avg_abs(dist)
            df["yr_avg"] = np.nan
            df["yr_avg_dist"] = np.nan
            df["yr_avg_abs(dist)"] = np.nan
            
            df["yr_avg"] = df.apply(
                lambda row: round(row["yr_max"] - row["yr_min"], 2) if (not row["yr_max"] == np.nan and not row["yr_min"] == np.nan) else np.nan
                , axis=1)
            df["yr_avg_dist"] = df.apply(
                lambda row: row["gw-level"] - row["yr_avg"] if not row["yr_avg"] == np.nan else np.nan
                , axis=1)
            df["yr_avg_abs(dist)"] = df.apply(
                lambda row: np.abs(row["yr_avg_dist"]) if not row["yr_avg_dist"] == np.nan else np.nan
                , axis=1)
            
                
            # add field: temp to add temperature (Celcius) to every monthly measurement if available   
            df["temp"] = np.nan
            
            mnth_temp_fpath = path.join(region_dir, "Grundwassertemperatur-Monatsmittel", "processed_data", "processed_Grundwassertemperatur-Monatsmittel-{0}.csv".format(mp_num))
            
            # not all mps have monthly temperature data, in this case we keep it as np.nan            
            if path.exists(mnth_temp_fpath):

                # time in both ds is 00:00:00, so I only care about the date
                df_mnth_temp = pd.read_csv(mnth_temp_fpath, sep=",")[["date", "temp"]]
                df_mnth_temp.set_index("date", inplace=True)
                
                df["temp"] = df.apply(
                    lambda row: df_mnth_temp.loc[row["date"]]["temp"] if row["date"] in df_mnth_temp.index else np.nan
                    , axis=1)
                
                # free resources
                del df_mnth_temp
            
            # drop ["time", "hour", "minute", "second"] cols since time is always 00:00:00
            del df["time"]
            del df["hour"]
            del df["minute"]
            del df["second"]
            
            # add mp attrs to each row
            res_mp = df_mps_all.query("hzbnr01==@mp_num").to_dict("records")[0] # dict of colname-val of first found row
            mp_attrs = ["x", "y", "dbmsnr", "hzbnr01", "region", "land_height", "mp_height",
                        "bottom_line", "t_measuring_depth"]
            attr_col_loc = 0 # add firsts attr at first col
            for attr in mp_attrs:
                df.insert(attr_col_loc, attr, res_mp[attr])
                attr_col_loc += 1 #add next attr at next col
            
            
            if debugging:
                print(df)
            
            # save summary df
            df_summary_path = path.join(summary_dir, "Summary-{0}.csv".format(mp_num))
            df.to_csv(df_summary_path, sep=",", index = False)            
                
            # free resources
            del df
            
            # if debugging, create summary file for only one mp 
            if debugging:
                break
            
        
    except Exception as ex:
        print("[Error]")
        print(traceback.format_exc())
        
    print("\t- Done!")
    
    return None

def process_region_files(region, region_dir):
    
    # process and manipulate region files into clean .csv files
    FILE_PROCESSING_DEBUGGING = False # False => process  all files     
    folder_names = [
        "Grundwasserstand-Jahresmaxima",
        "Grundwasserstand-Jahresminima",
        "Grundwasserstand-Monatsmittel",
        "Grundwassertemperatur-Monatsmittel"        
    ]
    
    for folder_name in folder_names:
        if not folder_name == "Grundwassertemperatur-Monatsmittel":
            val_col_name = "gw-level"
        else:
            val_col_name = "temp"
            
        process_region_sub_dir(region, region_dir, folder_name, val_col_name, debugging=FILE_PROCESSING_DEBUGGING)
        
    # create summary files on monthly base
    SUMMARY_Creating_DEBUGGING = False # False => create summary for all mps in region
    create_region_mps_summary(region, region_dir, debugging=SUMMARY_Creating_DEBUGGING)
    
    

In [13]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    for region_idx in range(len(regions)):
        region = regions[region_idx]
        region_dir = region_dirs[region_idx]
        process_region_files(region, region_dir)

> Processing Burgenland - Grundwasserstand-Jahresmaxima..
	- Done!
> Processing Burgenland - Grundwasserstand-Jahresminima..
	- Done!
> Processing Burgenland - Grundwasserstand-Monatsmittel..
	- Done!
> Processing Burgenland - Grundwassertemperatur-Monatsmittel..
	- Done!
> Creating Burgenland Summary..
	- Done!
> Processing Kärnten - Grundwasserstand-Jahresmaxima..
	- Done!
> Processing Kärnten - Grundwasserstand-Jahresminima..
	- Done!
> Processing Kärnten - Grundwasserstand-Monatsmittel..
	- Done!
> Processing Kärnten - Grundwassertemperatur-Monatsmittel..
	- Done!
> Creating Kärnten Summary..
	- Done!
> Processing Niederösterreich - Grundwasserstand-Jahresmaxima..
	- Done!
> Processing Niederösterreich - Grundwasserstand-Jahresminima..
	- Done!
> Processing Niederösterreich - Grundwasserstand-Monatsmittel..
	- Done!
> Processing Niederösterreich - Grundwassertemperatur-Monatsmittel..
	- Done!
> Creating Niederösterreich Summary..
	* Grundwasserstand-Jahresminima dropped 1 duplicate